**Mini Projeto de sistema de recomendação utilizando dados da Netflix**

In [ ]:
#Versão de Linguagem Python
from platform import python_version
print("A versão do python utilizada para este projeto é:", python_version())

A versão do python utilizada para este projeto é: 3.7.13


**Definição do problema**

Este projeto tem como objetivo desenvolver um sistema de recomendação utilizando dados da netflix com a finalidade de ajudar clientes a encontrar filmes e séries condizente com o seu gosto.

**Fonte de Dados**

Dataset: https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data?resource=download

**Importando pacotes e bibliotecas**

In [ ]:
!pip install -q -U watermark

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy
import sklearn
from scipy import sparse
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

# Formatação dos gráficos
matplotlib.use("nbagg")
plt.rcParams.update({"figure.max_open_warning": 0})
sns.set_style("whitegrid")


In [ ]:
# Versões dos pacotes utilizados neste projeto
%reload_ext watermark
%watermark -a "Diego Oliveira da Silva" --iversions

Author: Diego Oliveira da Silva

sklearn   : 0.0
numpy     : 1.21.5
pandas    : 1.3.5
seaborn   : 0.11.2
matplotlib: 3.2.2
IPython   : 5.5.0
scipy     : 1.4.1



**Carregando Dados**

In [ ]:
# Marca o início da execução de leitura dos arquivos
start = datetime.now() 

In [ ]:
# Se o arquivo não existir, criamos o arquivo em modo de escrita (W)
if not os.path.isfile("dados.csv"):
  # Cria e abre o arquivo para gravação
  dataset = open("dados.csv", mode = 'w')

  # Lista para as linhas dos arquivos
  linhas = list()

  # Nomes e caminhos dos arquivos
  arquivos = ["combined_data_1.txt"]
  # Loop por cada arquivo na lista de arquivos
  for arquivo in arquivos:
    # Print
    print("Lendo o arquivo {}..." .format(arquivo))
    # com o arquivo aberto, extraímos as linhas
    with open(arquivo) as f:
      # Loop por cada linha do arquivo
      for linha in f:
        # Deletamos o conteúdo da lista
        del linhas[:]
        #Divide as linhas do arquivo pelo caracter de final de linha
        linha = linha.strip()
        # Se encontramos "dois pontos" ao final da linha, fazemos replace removendo o caraxter,
        # Pos queremos apenas o id do filme
        if linha.endswith(":"):
          movie_id = linha.replace(":", "")
          # Se não, criamos uma list comprehension para fazer a separação colunas por vígulas
        else:
          # separa as colunas
          linhas = [x for x in linha.split(",")]

          # Usa o id do filme na posição de índice zero
          linhas.insert(0, movie_id)

          # Grava o resultado no novo arquivo
          dataset.write(",".join(linhas))
          dataset.write("\n")
      print("conluído.\n")
  dataset.close()



In [ ]:
# Imprime o tempo total
print("Tempo total para carregar os arquivos:", datetime.now() - start)

Tempo total para carregar os arquivos: 0:00:00.081529


In [ ]:
print("Criando o dataframe pandas a partir do arquivo dados.csv...")
df_netflix = pd.read_csv("dados.csv", sep = ",", names = ["movie","user", "rating", "date"])
df_netflix.date = pd.to_datetime(df_netflix.date)
print("Concluído.\n")

Criando o dataframe pandas a partir do arquivo dados.csv...
Concluído.



In [ ]:
# Ordenando o dataframe por data
print("Ordenando o dataframe por data...")
df_netflix.sort_values(by = "date", inplace = True)
print("Concluído.")

Ordenando o dataframe por data...
Concluído.


In [ ]:
# Shape
df_netflix.shape

(48708515, 4)

In [ ]:
# Visualizando os dados
df_netflix.head()

,movie,user,rating,date
19007335,5625,510180,4.0,1999-11-11
35551758,11313,510180,2.0,1999-11-11
9056171,1798,510180,5.0,1999-11-11
32021896,10774,510180,3.0,1999-11-11
47201120,15057,510180,5.0,1999-11-11


**Análise exploratória dos dados**

In [ ]:
# Resumo dos dados
print("Resumo dos dados")
print("-" * 50)
print("Número Total de Filmes: ", len(np.unique(df_netflix.movie)))
print("Número Total de Usuários: ", len(np.unique(df_netflix.user)))
print("Número Total de Avaliações: ", df_netflix.shape[0])

Resumo dos dados
--------------------------------------------------
Número Total de Filmes:  8443
Número Total de Usuários:  478125
Número Total de Avaliações:  48708515


In [ ]:
# Verificando a média das avaliações
df_netflix.describe()['rating']

count    4.870851e+07
mean     3.611033e+00
std      1.084796e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [ ]:
# Verificando se temos valores ausente
sum(df_netflix.isnull().any())

2

In [ ]:
# Verificando se temos valores duplicados (para esse caso não considera a data)
sum(df_netflix.duplicated(["movie", "user", "rating"]))

0

In [ ]:
#Criando data set com dados de treino
# Sendo assim, precisa executar todo o processo de carga novamente cada vez que executar este notebook
if not os.path.isfile("dados_treino.csv"):
  df_netflix.iloc[:int(df_netflix.shape[0] * 0.80)].to_csv("dados_treino.csv", index = False)

In [ ]:
# Criando um data set com dados de teste
# Sendo assim, não precisa executar todo o processo de carga novamente cada vez que executar este notebook
if not os.path.isfile("dados_teste.csv"):
  df_netflix.iloc[int(df_netflix.shape[0] * 0.80):].to_csv("dados_teste.csv", index = False)

In [ ]:
# Deletando o dataframe origital para liberar memória
del df_netflix

In [ ]:
#Agora carregamos os arquivos em dataframes do pandas
df_netflix_treino = pd.read_csv("dados_treino.csv", parse_dates = ["date"])
df_netflix_teste = pd.read_csv("dados_teste.csv")

In [ ]:
# Resumo dados de treino
print("Resumo dos dados de treino")
print("-" * 50)
print("Número Total de Filmes: ", len(np.unique(df_netflix_treino.movie)))
print("Número total de usuários: ", len(np.unique(df_netflix_treino.user)))
print("Número total de avaliações: ", df_netflix_treino.shape[0])

Resumo dos dados de treino
--------------------------------------------------
Número Total de Filmes:  8272
Número total de usuários:  401104
Número total de avaliações:  38966812


In [ ]:
# Resumo dos dados de teste
print("Resumo dos dados de teste")
print("-" * 50)
print("Número total de Filmes: ", len(np.unique(df_netflix_teste.movie)))
print("Número total de usuários: ", len(np.unique(df_netflix_teste.user)))
print("Número total de avaliações: ", df_netflix_teste.shape[0])

Resumo dos dados de teste
--------------------------------------------------
Número total de Filmes:  8438
Número total de usuários:  323963
Número total de avaliações:  9741703


In [ ]:
# Função para ajuste das unidades de medida
def ajusta_unidades(num, units = "M"):
  units = units.lower()
  num = float(num)
  if units == "K":
    return str(num / 10**3) + " K"
  elif units == "m":
    return str(num / 10**6) + " M"
  elif units == "b":
    return str(num / 10**9) + " B"

In [ ]:
# Supress warnings
import sys
import warnings
if not sys.warnoptions:
  warnings.simplesfilter("ignore")


In [ ]:
# Plot
fig, ax = plt.subplots()
plt.title("distribuição das avaliações nos dados de treino", fontsize = 15)
sns.countplot(df_netflix_treino.rating)
ax.set_yticklabels([ajusta_unidades(item, "M") for item in ax.get_yticks()])
ax.set_ylabel("Número de avaliações (em milhões)")
plt.show()

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


**Verificando se o dia da semana tem influência na avaliação do usuário**

In [ ]:
# Paramentro para evitar warning devido ao alto volume de dados
pd.options.mode.chained_assignment = None

In [ ]:
# Extrai o dia da semana e grava em uma nova coluna
df_netflix_treino["dia_semana"] = df_netflix_treino["date"].dt.strftime("%A")
df_netflix_treino.head()

,movie,user,rating,date,dia_semana
0,5625,510180,4.0,1999-11-11,Thursday
1,11313,510180,2.0,1999-11-11,Thursday
2,1798,510180,5.0,1999-11-11,Thursday
3,10774,510180,3.0,1999-11-11,Thursday
4,15057,510180,5.0,1999-11-11,Thursday


In [ ]:
#Plot
fig, ax = plt.subplots()
sns.countplot(x = "dia_semana", data = df_netflix_treino, ax = ax)
plt.title("Número de avaliações por dia da semana")
plt.ylabel("Total de avaliações")
plt.xlabel("")
ax.set_yticklabels([ajusta_unidades(item, "M") for item in ax.get_yticks()])
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
# Média de avaliações por dia da semana
media_dia_semana = df_netflix_treino.groupby(by = ["dia_semana"])["rating"].mean()
print("Média de avaliações")
print("-"*30)
print(media_dia_semana)
print("\n")

Média de avaliações
------------------------------
dia_semana
Friday       3.598141
Monday       3.587838
Saturday     3.600846
Sunday       3.602084
Thursday     3.591507
Tuesday      3.585715
Wednesday    3.592052
Name: rating, dtype: float64




**Verificando avalições no decorrer do tempo**

In [ ]:
fig = plt.figure(figsize = plt.figaspect(.45))
ax = df_netflix_treino.resample("m", on = 'date')["rating"].count().plot()
ax.set_title("Número de avaliações por mês nos dados de Treino")
plt.xlabel("Mês")
plt.ylabel("Número de avaliações por mês")
ax.set_yticklabels([ajusta_unidades(item, "M") for item in ax.get_yticks()])
plt.show()

<IPython.core.display.Javascript object>

**Função de densidade de probabilidade e distribuição acumulada**

In [ ]:
# Número de avaliações por usuário
num_aval_por_user = df_netflix_treino.groupby(by = 'user')['rating'].count().sort_values(ascending = False)
num_aval_por_user.head()

user
305344     8120
2439493    7548
387418     7305
1639792    4625
1461435    4511
Name: rating, dtype: int64

In [ ]:
num_aval_por_user.describe()

count    401104.000000
mean         97.148899
std         140.012203
min           1.000000
25%          17.000000
50%          45.000000
75%         121.000000
max        8120.000000
Name: rating, dtype: float64

In [ ]:
# Plot
fig = plt.figure(figsize = plt.figaspect(.45))
ax1 = plt.subplot(121)
sns.kdeplot(num_aval_por_user, shade = True, ax = ax1)
plt.xlabel("Número de avaliações por usuário")
plt.title("PDF - Função de densidade de probabilidade")
ax2 = plt.subplot(122)
sns.kdeplot(num_aval_por_user, shade = True, cumulative = True, ax = ax2)
plt.xlabel("Número de avaliações por usuário")
plt.title("CDF - Função de densidade acumulada")
plt.show()

<IPython.core.display.Javascript object>

**Criando uma matriz esparsa de Treino**


In [ ]:
if os.path.isfile("matriz_esparsa_treino.npz"):
  matriz_esparsa_treino = sparse.load_npz("matriz_esparsa_treino.npz")
  print("Matriz Carregada.")
else:
  matriz_esparsa_treino = sparse.csr_matrix((df_netflix_treino.rating.values, (df_netflix_treino.user.values,
                                                                               df_netflix_treino.movie.values)),)
  print("Matriz criada. O shape é: (user, movie): ", matriz_esparsa_treino.shape)
sparse.save_npz("matriz_esparse_treino.npz", matriz_esparsa_treino)
print("Matriz Salva em disco.")

Matriz criada. O shape é: (user, movie):  (2649430, 15210)
Matriz Salva em disco.


In [ ]:
# Calculando a densidade da matriz
linhas, colunas = matriz_esparsa_treino.shape
elementos_nao_zero = matriz_esparsa_treino.count_nonzero()
print("Esparsidade da matriz de treino: {} %".format( (1 - (elementos_nao_zero / (linhas * colunas))) * 100) )

Esparsidade da matriz de treino: 99.90330295276468 %


**Criação da matriz esparsa de teste**

In [ ]:
if os.path.isfile("matriz_esparsa_teste.npz"):
  matriz_esparsa_teste = sparse.load_npz("matriz_esparse_teste.npz")
  print("Matriz carregada.")
else:
  matriz_esparsa_teste = sparse.csr_matrix((df_netflix_teste.rating.values, (df_netflix_teste.user.values,
                                                                               df_netflix_teste.movie.values)),)
  print("Matriz criada. O shape é: (user, movie): ", matriz_esparsa_teste.shape)
  sparse.save_npz("matriz_esparse_teste.npz", matriz_esparsa_teste)
  print("Matriz Salva em disco.")

Matriz criada. O shape é: (user, movie):  (2649430, 15210)
Matriz Salva em disco.


In [ ]:
# Calculamos a esparsidade da matriz
linhas, colunas = matriz_esparsa_teste.shape
elementos_nao_zero = matriz_esparsa_teste.count_nonzero()
print("Esparsidade da matriz de teste: {} %".format( (1 - (elementos_nao_zero / (linhas * colunas))) * 100) )

Esparsidade da matriz de teste: 99.97582573819118 %


**Calculando a média global de todas as avaliações de filmes**

In [ ]:
#Abaixo o cálculo da média global de todas as avaliações dos usuários
medias_treino = dict()
medias_treino_global = matriz_esparsa_treino.sum() / matriz_esparsa_treino.count_nonzero()
medias_treino["global"] = medias_treino_global
medias_treino

{'global': 3.5928531695125585}

**Construção de uma função para calcular a média de avaliações**

In [ ]:
#Função de cálculo da média
def calcula_media_avaliacoes(sparse_matrix, of_users):
  # média usuários / eixos 
  # 1 = eixo do usuário
  # 0 = eixo do filme
  
  ax = 1 if of_users else 0

  # Soma
  sum_of_ratings = sparse_matrix.sum(axis = ax).A1

  # Matriz booleana de avaliações (Se um usuário avaliou um filme ou não)
  is_rated = sparse_matrix != 0

  # Número de avaliações de cada usuário ou filme
  no_of_ratings = is_rated.sum(axis = ax).A1

  # Máxima de usuários e filmes na matriz esparsa
  u, m = sparse_matrix.shape

  # Criando um dicionário de usuários e suas avaliações médias
  media_aval = {i: sum_of_ratings[i] / no_of_ratings[i] for i in range(u if of_users else m) if  no_of_ratings[i]!= 0}  

  # Retorna o dicionário de média de avaliações 
  return media_aval

**Abaixo calcularemos a média de avaliação por usuário**

In [ ]:
# média de avaliações de usuários
medias_treino["user"] = calcula_media_avaliacoes(matriz_esparsa_treino, of_users = True)


In [ ]:
# Visualiza o dicionário
medias_treino

{'global': 3.5928531695125585,
 'user': {6: 3.5675675675675675,
  7: 4.030444964871195,
  10: 3.4423076923076925,
  25: 3.5,
  33: 3.9,
  42: 3.925925925925926,
  59: 3.6555555555555554,
  79: 3.5391061452513966,
  83: 3.923076923076923,
  87: 3.4909090909090907,
  94: 2.526315789473684,
  97: 3.0865800865800868,
  131: 3.5,
  134: 4.700879765395895,
  142: 3.5,
  149: 4.571428571428571,
  158: 3.6,
  168: 4.5,
  169: 3.72,
  178: 3.0,
  183: 3.823529411764706,
  188: 3.433734939759036,
  189: 3.0,
  192: 3.4,
  195: 3.7218543046357615,
  199: 3.982142857142857,
  201: 3.6646706586826348,
  242: 2.7777777777777777,
  247: 3.9565217391304346,
  248: 3.7777777777777777,
  261: 2.933333333333333,
  265: 3.671814671814672,
  266: 4.182608695652174,
  267: 3.238095238095238,
  268: 4.101694915254237,
  283: 3.424107142857143,
  291: 3.466666666666667,
  296: 4.235294117647059,
  298: 3.8523489932885906,
  299: 3.625,
  301: 4.094117647058823,
  302: 3.288,
  304: 3.8,
  305: 4.0597014925373

**Calculo da média de avaloações por filme**

In [ ]:
# média de avaliações por filme
medias_treino["movie"] = calcula_media_avaliacoes(matriz_esparsa_treino, of_users = False)

**Calculando matriz de similaridade entre usuarios**

In [ ]:
# Função de cálculo de similaridade
def calcula_similaridade_usuarios(sparse_matrix, 
                                 compute_for_few = False, 
                                 top = 100, 
                                 verbose = False, 
                                 verb_for_n_rows = 20,
                                 draw_time_taken = True):
    
    # Variáveis de controle
    no_of_users, _ = sparse_matrix.shape
    row_ind, col_ind = sparse_matrix.nonzero()
    row_ind = sorted(set(row_ind)) 
    time_taken = list()
    rows, cols, data = list(), list(), list()
    if verbose: print("Calculando top", top, "similaridades para cada usuário...")
    start = datetime.now()
    temp = 0
    
    # Loop pela matriz
    for row in row_ind[:top] if compute_for_few else row_ind:
        temp = temp + 1
        prev = datetime.now()
        
        # Calculando a similaridade de cosseno
        sim = cosine_similarity(sparse_matrix.getrow(row), sparse_matrix).ravel()
        top_sim_ind = sim.argsort()[-top:]
        top_sim_val = sim[top_sim_ind]
        rows.extend([row]*top)
        cols.extend(top_sim_ind)
        data.extend(top_sim_val)
        time_taken.append(datetime.now().timestamp() - prev.timestamp())
        
        if verbose:
            if temp%verb_for_n_rows == 0:
                print("Cálculo concluído para {} usuários [  tempo total : {}  ]".format(temp, datetime.now()-start))
            
    if verbose: print('Criação de matriz esparsa a partir das semelhanças computadas...')    
        
    if draw_time_taken:
        plt.plot(time_taken, label = 'Tempo de cálculo de cada usuário')
        plt.plot(np.cumsum(time_taken), label = 'Tempo Total')
        plt.legend(loc = 'best')
        plt.xlabel('Usuário')
        plt.ylabel('Tempo (segundos)')
        plt.show()
        
    return sparse.csr_matrix((data, (rows, cols)), shape = (no_of_users, no_of_users)), time_taken  

In [ ]:
#Cauculo de similaridade
#Marca o ínicio
start = datetime.now()

# Calcula a similaridade
matriz_esparsa_user, _ = calcula_similaridade_usuarios(matriz_esparsa_treino,
                                                      compute_for_few = True,
                                                      top = 100,
                                                      verbose = True)

print("Tempo Total de Processamento:", datetime.now() - start)

Calculando top 100 similaridades para cada usuário...
Cálculo concluído para 20 usuários [  tempo total : 0:00:40.567008  ]
Cálculo concluído para 40 usuários [  tempo total : 0:01:15.663813  ]
Cálculo concluído para 60 usuários [  tempo total : 0:01:49.909017  ]
Cálculo concluído para 80 usuários [  tempo total : 0:02:24.868347  ]
Cálculo concluído para 100 usuários [  tempo total : 0:03:00.271182  ]
Criação de matriz esparsa a partir das semelhanças computadas...
Tempo Total de Processamento: 0:03:03.683996


**Redução de dimensionalidade de truncatedSVD**

In [ ]:
# Redução de dimencionalidade
start = datetime.now()

# Cria o objeto TruncatedSVD reduzindo a dimensionalidade para 500 dimensões
netflix_svd = TruncatedSVD(n_components = 500, algorithm = 'randomized', random_state = 15)

# Aplica o TruncatedSVD
trunc_svd = netflix_svd.fit_transform(matriz_esparsa_treino)

print("Tempo Total de Processamento:", datetime.now() - start)

In [ ]:
# Variância explicada pelo componentes
expl_var = np.cumsum(netflix_svd.explained_variance_ratio_)


In [ ]:
# Plot

fig, (ax1) = plt.subplots(nrows = 1, nools = 1, figsize - plt.figaspect(.45))
ax1.set.xlabel("Variância explicada", fontsize = 15)
ax1.plot(expl_var)

ind = [1, 2, 4, 8, 20, 60, 100, 200, 300, 400, 500]
ax1.scarted(x = [i - 1 for i in ind], y = expl_var[[i - 1 for i in ind]], c = "#ee4422")

for i in ind:
  ax1.anotate(s = "({}, {}".format(i, np.round(expl_var[i - 1], 2)), xy = (i - 1, expl_var[i - 1]),
              xytext = ( i + 20, expl_var[i - 1] - 0.01),fontweight = "bold")
plt.show()

In [ ]:
# Projetando matriz no espaço de 500 dimensões
start = datetime.now()
trunc_matrix = matriz_esparsa_treino.dot(netflix_svd.components_.T)
print("Tempo de processamento: ", datetime.now() - start)

In [ ]:
if not os.path.isfile("matriz_esparsa_user_truncada.npz"):
  matriz_esparsa_user_truncada = sparse.csr.matrix(trunc_matrix)
  sparse.save_npz("matriz_esparsa_user_truncada", matriz_esparsa_user_truncada)
else:
  matriz_esparsa_user_truncada = sparse.load.npz("matriz_esparsa_user_truncada.npz")

**Usando a matriz truncada para o cálculo de similaridade**

In [ ]:
start = datetime.now()

# Calculando a similaridade

trunc_sim_matrix, _ = calcula_similaridade_usuarios(matriz_esparsa_user_truncada,
                                                    compute_for_few = True,
                                                    top = 50,
                                                    verbose = True)

print("Tempo de processamento: ", datetime.now() - start)

**Sistema de similaridade de Filmes**

In [ ]:
start = datetime.now()

#Cria caso não exista
if not os.path.isile("matriz_esparsa_filme.npz"):
  matriz_esparsa_filme = cosine_similarity(x = matriz_esparsa_treino.T, dense_output = False)
  print("Matriz Criada.")
  sparse.save_npz("matriz_esparsa_filme.npz", matriz_esparsa_filme)
  print("Matriz Salva em disco.")
else:
  matriz_esparsa_filme = sparse_load.npz("matriz_esparsa_filme.npz")
  print("Matriz carregada.")
print("Tempo de processamento: ", datatime.now() - start)

In [ ]:
# Extra os ids dos filmes
movie_ids = np.unique(matriz_esparsa_filme.nonzero()[1])


In [ ]:
# Calculando as similaridade dos filmes de acordo com as avaliações dos usuários
start = datetime.now()

# Cria dicionário para armazenar as similaridade 
filmes_similares = dict()

#Loop pelos ids dos filmes
for movie in movie_ids:
  filmes_sim = matriz_esparsa_filme[movie].toarray().ravel().argsort()[::-1][1:]
  filmes_similares[movie] = filmes_sim[:100]

print("Tempo de processamento: " datetime.now() - start)


In [ ]:
# Encontrando filmes de acordo com as similaridades dos filmes
titulos_filmes = pd.read.csv("movie.titles.csv",
                             sep = ",",
                             name = ["ID_filme", "Ano_Lancamento", "Titulo"],
                             verbose = True,
                             index_col = "ID_Filme",
                             encoding = "ISO-8859-1")

In [ ]:
#Verificando filmes similares
ID_filme = int(input("Digite o Id do filme que deseja saber a similaridade: "))

In [ ]:
print("Filme: ", titulos_filme.loc[ID_filme].values[1])
print("Total de Avaliações de usuários = {}".format(matriz_esparsa_treino[:,ID_filme].getnnz()))
print("Encontramos {} Filmes similares a este".format(matriz_esparsa_filme))

In [ ]:
#filmes similares
similarities = matriz_esparsa_filme[ID_filme].toarray().ravel()
similar_indices = similarities.argsort()[::-1][1:]
similarities[similar_indices]
sim_indices = similarities.argsort()[::-1][1:]

In [ ]:
#Plot
fig = plt.figure(figsize = pltaspect(.45))
plt.plot(similarities[sim_indices], label = "Todas as Avaliações")
plt.plot(similarities[sim_indices[::100]], label = "top 100 FIlmes similares")
plt.title("Filmes similares ao filme {}".format(ID_FIlme), fontsize = 25)
plt.xlabel("Filmes", fontsize = 15)
plt.ylabel("Similares de cosseno", fontsize = 15)
plt.legend()
plt.show()